# Homework 8.3 [SOLO]: Modeling and parameter estimation for Boolean data (35 pts)

<hr>

In this problem, we will work with data of the True/False type. Lots of data sets in the biological sciences are like this. For example, we might look at a certain mutation in *Drosophila* that affects development and we might check whether or not eggs hatch.

The data we will use comes from an experiment we have done for the last few years in [Bi 1x](http://bi1x.caltech.edu/) here at Caltech. The experiment was developed by Meaghan Sullivan in collaboration with Ravi Nath. Sophie Walton further developed strains that were used in the 2019 experiment, which we study here. We studied a neural circuit in *C. elegans* using optogenetics.

A neural circuit is a series of interconnected neurons that create a pathway to transmit a signal from where it is received to where it causes a behavioral response in an animal.  An example is the neural circuit involved in reversals in *C. elegans*.  This circuit consists of three types of neurons: sensory neurons receive stimuli from the environment, command interneurons integrate information from many sensory neurons and pass a signal to the motor neurons, and motor neurons that control worm behavior, such as reversals.

There are six non-motor neurons acting in a circuit that responds to environmental cues and triggers a reversal, a shown in the figure below (based on [Schultheis et. al. 2011](https://doi.org/10.1371/journal.pone.0018766)).  These include four sensory neurons (ALM, AVM, ASH, and PLM) and two interneurons (AVD and AVA).  Each sensory neuron is sensitive to a different type of stimulus.  For example, the sensory neuron we are studying (ASH) is sensitive to chemosensory stimuli such as toxins, while another neuron (PLM) is sensitive to mechanical stimuli (touch) in the posterior part of the worm's body.  The sensory neurons send signals that are integrated by the two command interneurons (AVA and AVD).  Each sensory neuron can provide an impulse to the command interneurons at any time.  In order for the command interneuron to fire and activate motor neurons, the sum of the stimuli at any point in time must exceed a certain threshold.  Once the stimuli from one or more sensory neurons has induced an action potential in a command interneuron, that signal is passed to motor neurons which will modulate worm behavior.

<div style="margin: auto; width: 400px">
    
![Reversal neural network](reversal_neural_network.png)
    
</div> 

In the experiment, we used optogenetics to dissect the function of individual neurons in this circuit.  We worked with two optogenetic worm strains.  The ASH strain has channelrhodopsin (ChR2, represented by a red barrel in the figure above) expressed only in the ASH sensory neuron.  When we shine blue light on this strain, we should activate the ChR2, which will allow sodium and calcium cations to flow into the neuron, simulating an action potential. We want to quantify how robustly this stimulation will cause the worm to exhibit aversion behavior and reverse.  

We also studied an AVA strain that has channelrhodopsin expressed only in the AVA command interneuron.  Our goal is to quantify the effects of stimulating this neuron in terms of reversals compared to the ASH neuron and to wild type.

The True/False data here are the whether or not the worms undergo a reversal. Here is what the students observed in 2019, the last year we did this experiment.

|Strain|Year|Trials|Reversals|
|:---:|:---:|:---:|:---:|
|WT|2019|50|6|
|ASH|2019|52|15|
|AVA|2019|53|40|

<!--

|WT|2017|55|7|
|ASH|2017|54|18|
|AVA|2017|52|28|
|WT|2016|36|6|
|ASH|2016|35|12|
|AVA|2016|36|30|
|WT|2015|35|0|
|ASH|2015|35|9|
|AVA|2015|36|33|
-->

Our goal is to estimate $\theta$, the probability of reversal for each respective strain.

**a)** Propose a generative model for worm reversals. It should include the parameter $\theta$.

**b)** If possible, derive an analytical expression for the maximum likelihood estimate of $\theta$. If you have the expression, compute the MLE for $\theta$ for each of the three strains. If not, numerically compute the MLE for $\theta$ for each of the three strains.

**c)** Compute confidence intervals for your estimates of $\theta$.

**d)** Comment on the results and conduct any other analyses you think may help you understand the results of this experiment.

In [1]:
import os, sys
import numpy as np
import warnings
import scipy
import scipy.optimize
import tqdm
from scipy.stats import multivariate_normal

**a)** Propose a generative model for worm reversals. It should include the parameter $\theta$.

I proposed that the worm reversal will happen in a binomial distribution. The number of trial is N and the number selected is n and $\theta$ is the probility of the worm reverse. The null hypothesis is that all the $\theta$ are the same for all strains

**b)** If possible, derive an analytical expression for the maximum likelihood estimate of $\theta$. If you have the expression, compute the MLE for $\theta$ for each of the three strains. If not, numerically compute the MLE for $\theta$ for each of the three strains.

\begin{align}
    P(n,N,\theta)=\frac{N!}{n!(N-n)!} \theta ^n (1- \theta)^(N-n)
\end{align}
since the factorical term is only a constant which doesn't affect the MLE approximation, we will replace such as a constant k 
\begin{align}
    L(n,N,\theta)=k+(N-n)log(1-\theta)+nlog(\theta)
\end{align}
thus we take the derivative of the likelihood function and set to 0 to find the local minimum
\begin{align}
    \frac{d(L(n,N,\theta))}{d\theta}=-\frac{N-n}{1-\theta}+\frac{n}{\theta}
\end{align}
\begin{align}|
    0=-\frac{N-n}{1-\theta}+\frac{n}{\theta}
\end{align}
\begin{align}
    \frac{N-n}{1-\theta}=\frac{n}{\theta}
\end{align}
\begin{align}
    \theta=\frac{n}{N}
\end{align}

**c)** Compute confidence intervals for your estimates of $\theta$.

In [2]:
WT=np.ones(50)
WT[:44]=np.zeros(44)
ASH=np.zeros(52)
ASH[:15]=np.ones(15)
AVA=np.zeros(53)
AVA[:40]=np.ones(40)

In [29]:
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    generate_random=np.random.choice(data, np.random.randint(len(data)))
    return len(generate_random), np.sum(generate_random ==1)

def theta_r(N,n):
    """
    Compute plug-in estimate for the bivariate correlation coefficient.
    """
    return (
        n/N
    )
def draw_bs_theta(data, size=1):
    """
    Draw bootstrap pairs replicates.
    """
    out = np.empty(size)

    for i in range(size):
        out[i] = theta_r(*draw_bs_sample(data))

    return out

In [36]:
#draw_bs_sample(WT)
WT_theta=draw_bs_theta(WT,50)
conf_int_theta_WT = np.percentile(WT_theta, [2.5, 97.5])
ASH_theta=draw_bs_theta(ASH,50)
conf_int_theta_ASH = np.percentile(ASH_theta, [2.5, 97.5])
AVA_theta=draw_bs_theta(AVA,50)
conf_int_theta_AVA = np.percentile(AVA_theta, [2.5, 97.5])

print(
    """
θ for WT int:   [{0:.2f}, {1:.2f}]
θ for ASH int: [{2:.2f}, {3:.2f}]
θ for AVA int: [{4:.2f}, {5:.2f}]
""".format(
        *tuple(conf_int_theta_WT), *tuple(conf_int_theta_ASH),  *tuple(conf_int_theta_AVA)
    )
)


θ for WT int:   [0.00, 0.27]
θ for ASH int: [0.02, 0.56]
θ for AVA int: [0.50, 0.99]



**d)** Comment on the results and conduct any other analyses you think may help you understand the results of this experiment.

I think the results of the confidence interval of $\theta$ make sense because the higher the reversal rate the higher the \theta should be for binomial distribution, thus AVA has a higher $\theta$ than ASH and ASH has a higher $\theta$ than WT. Different strain of worms has different $\theta$, we reject the null hypothesis